In [1]:
%pip install scikeras

In [2]:
import os
import json
import numpy as np
import tensorflow as tf
import time
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam




2024-10-29 15:31:39.967011: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 15:31:40.279265: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 15:31:40.365076: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 15:31:40.383505: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 15:31:40.425881: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Cargar el dataset desde el archivo .npy
path2 = '../DatasetVGG19/train.npy'

In [4]:
data_train = np.load(path2)

In [5]:

def unpack(array):
    x = array[:,:-2]
    y = array[:,-2:]
    y = np.argmax(y, axis=1)
    return x,y

def model_score_tf(X,y,model,dataset):
    test_loss, test_accuracy = model.evaluate(X, y)
    print(f"Precisión del modelo {dataset}: {test_accuracy * 100:.2f}%")
    return test_accuracy

def writeJson(tiempo,accu,metod):
    nuevo_registro = {
        'Tipo Ejecucion' : "GPU",
        'Metodo' : metod,
        'Epochs':20,
        'tiempo_ejecucion': tiempo,
        'accuracy' : accu
    }
    nombre_archivo = '../GPU_tiempos.json'

    # Leer el contenido existente, si el archivo ya existe
    if os.path.exists(nombre_archivo):
        with open(nombre_archivo, 'r') as archivo:
            try:
                registros = json.load(archivo)
            except json.JSONDecodeError:
                registros = []
    else:
        registros = []

    # Agregar el nuevo registro
    registros.append(nuevo_registro)

    # Guardar los registros actualizados en el archivo JSON
    with open(nombre_archivo, 'w') as archivo:
        json.dump(registros, archivo, indent=4)

In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

I0000 00:00:1730233907.436791   22770 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730233907.722023   22770 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730233907.722128   22770 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


# MLPClassifier

In [7]:
X_train,y_train = unpack(data_train)

# Convertir las etiquetas a formato one-hot para TensorFlow
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)

In [8]:
def create_model(learning_rate=0.01, hidden_activation='relu', output_activation='sigmoid', layers=(500, 200, 2)):
    layers_arr = []

    for i in range(len(layers)):
        if i == 0:
            layers_arr.append(Dense(layers[i], activation=hidden_activation, input_shape=(1000,)))
        elif len(layers)-1:
            layers_arr.append(Dense(layers[i], activation=output_activation))
        else:
            layers_arr.append(Dense(layers[i], activation=hidden_activation))

    model = Sequential(layers_arr)
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                loss='binary_crossentropy', 
                metrics=['accuracy'])
    return model

In [9]:
print("Entrenamiento MLP Classifier")

param_grid = {
    'layers': [(32, 16, 2), (32,  16, 8, 2)],
    'learning_rate': [0.001]
}
model = KerasClassifier(build_fn=create_model,learning_rate=0.01,  epochs=20, batch_size=32, verbose=0, hidden_activation='relu', output_activation='sigmoid', layers=(500, 200, 2))
inicio = time.time()

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=1)
grid_result = grid.fit(X_train,y_train)

fin = time.time()
tiempo_ejecucion_MLPC = fin - inicio

print(f"El tiempo de ejecucion fue de {tiempo_ejecucion_MLPC:.2f} segundos")

/home/danielxf/.local/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/danielxf/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Entrenamiento MLP Classifier


I0000 00:00:1730233907.876898   22770 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730233907.877165   22770 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730233907.877207   22770 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730233908.159643   22770 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730233908.159914   22770 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-29

El tiempo de ejecucion fue de 103.80 segundos


In [10]:
writeJson(tiempo_ejecucion_MLPC,grid_result.best_score_,"MLPC")

# Tiempos

In [11]:
print(f"Tiempo MLPC = {tiempo_ejecucion_MLPC:.2f} segundos")

Tiempo MLPC = 103.80 segundos
